# Capítulo 19: Dinâmica molecular e análise de trajetória

Livro: Aprendendo Química com Python, Rodrigo Q. de Albuquerque & Gerd B. da Rocha, 2021, Amazon Book.

E-Mail: learn.chem.python@gmail.com

### Instruções:

1. Recomendado o uso do Linux;

2. Instalar o Python a partir do gerenciador de pacote Anaconda: https://www.anaconda.com/products/individual-d

3. Atualiza os pacotes com conda: conda update -n ENVIRONMENT --all

4. Instala o AMBERTools --> conda install -c conda-forge ambertools

5. Instala o PyTraj: conda install -c conda-forge pytraj

6. Instala o NGLView: conda install -c conda-forge nglview

7. Verifica se tudo foi instalado corretamente: conda list

8. Caso esteja usando o Jupyter-Notebook, antes de carregá-lo executar o comando:
    jupyter-nbextension enable nglview --py --sys-prefix

# Biblioteca NGL Viewer

In [ ]:
# Verificando a versão da NGL Viewer instalada
import nglview as nv
print('Versão do NGLView --> ', nv.__version__)

In [ ]:
# Carregando a estrutura da proteína 2CI2 do banco de dados PDB (https://www.rcsb.org/structure/2CI2) 
view1 = nv.show_pdbid("2ci2")  
view1

In [ ]:
# Para modificar a representação do objeto "view1" e salvar a figura
view1.parameters = dict(theme='light')
view1.download_image(filename='2ci2.png', factor=15, trim=True)

In [ ]:
# Para adicionar superfícies no objeto "view1"
view1.add_surface(selection="protein", opacity=0.3)

# Realizando a dinâmica molecular

### Para instalar o pacote AMBERTools

conda install -c conda-forge ambertools

In [ ]:
# Forma de escrever um arquivo direto do Jupyter-Notebook
# Montando a estrutura do peptídeo e sua topologia
# OBS: Não usar comentários na caixa que escreve o arquivo "leap.inp"

In [ ]:
%%file leap.inp
source leaprc.protein.ff14SB
seq = sequence{NALA ALA ALA ALA ALA CALA}
set default PBradii mbondi3 
saveamberparm seq peptide.top peptide_ini.crd
savepdb seq peptide_ini.pdb 
quit

In [ ]:
# Forma de executar um comando do sistema operacional (Linux)
!$AMBERHOME/bin/tleap -f leap.inp > leap.log

In [ ]:
# Monta o input para minimizar a energia
# OBS: Não usar comentários na caixa que escreve o arquivo "min_implicit.in"

In [ ]:
%%file min_implicit.inp
Minimizacao da energia
&cntrl
  imin=1, maxcyc=5000, ncyc=1000,
  cut=999., igb=8, ntb=0, ntpr=100
&end

In [ ]:
# Para otimizar a geometria do peptídeo (ALA)6
!$AMBERHOME/bin/sander -O -i min_implicit.inp -p peptide.top -c peptide_ini.crd -r min_implicit.rst7

In [ ]:
# Importar a PyTraj
import pytraj as pt
print('Versão do PyTraj --> ', pt.__version__)

In [ ]:
# Carrega a estrutura otimizada com a PyTraj
ref_implicit_min = pt.load('min_implicit.rst7', top='peptide.top')

# Visualizar a estrutura 
view_min_implicit = nv.show_pytraj(ref_implicit_min)
view_min_implicit.add_ball_and_stick()
view_min_implicit.add_cartoon()
view_min_implicit.parameters = dict(camera_type='perpective', background_color='black',fog_near=60, clip_dist=0)
view_min_implicit

In [ ]:
view_min_implicit.render_image()

In [ ]:
view_min_implicit._display_image()

In [ ]:
view_min_implicit.parameters = dict(theme='light')
view_min_implicit.download_image(filename='hexa_ala_min_imp.png', factor=8, trim=True)

In [ ]:
# Monta input para dinâmica molecular NVT
# OBS: Não usar comentários na caixa que escreve o arquivo "md_implicit.in"

In [ ]:
%%file md_implicit.inp
MD NVT, 500 ps
&cntrl
    imin = 0, nstlim = 250000, dt = 0.002, ntf = 2, nscm = 1000, 
    ntx = 1, irest = 0, ig = -1, ntc = 2, temp0 = 300.0, gamma_ln = 1.0, 
    ntt=3, tempi = 300.0, ntwr = 500, ntpr = 500, ntb = 0, ntwx = 500, 
    ntwe = 0, cut = 999.0, igb = 8, ioutfm = 1,
&end

In [ ]:
# Para executar a dinâmica moleculare NVT para o peptídeo (ALA)6
!$AMBERHOME/bin/sander -O -i md_implicit.inp -o md_implicit.out -p peptide.top -c min_implicit.rst7 -r md_implicit.rst -x md_implicit.nc

In [ ]:
# Observar a execução da dinâmica molecular
!tail -n 20 md_implicit.out

In [ ]:
# Carregar a trajetória no objeto "traj_implicit"
traj_implicit = pt.load('md_implicit.nc', top='peptide.top')
traj_implicit

# Analisando a trajetória simulada

### Alinhamento dos frames da trajetória simulada

In [ ]:
# Comandos para alinhar a trajetória pelo frame de referência
traj_implicit_aligned = pt.align(traj_implicit, ref=0)

# Comandos para gerar uma visualização da trajetória de dinâmica molecular
view_traj_implicit = nv.show_pytraj(traj_implicit_aligned)

# Comandos para gerar uma representação para a trajetória carregada
view_traj_implicit.add_ball_and_stick()
view_traj_implicit.add_cartoon()
view_traj_implicit.parameters = dict(camera_type='perpective', background_color='black',fog_near=60, clip_dist=0)

# Mostra a animaçãona tela
view_traj_implicit

## Cálculo de RMSD

In [ ]:
# Importar a biblioteca Matplotlib
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
# RMSD em solvente implícito
rmsd_implicit = pt.rmsd(traj_implicit, ref=0)

# Montagem do gráfico do RMSD ao longo da dinâmica molecular
plt.figure(figsize=(8, 6))
plt.plot(rmsd_implicit)
plt.title('Dinâmica Molecular da $(ALA)_6$ em solvente implícito', fontsize=16, pad = 14)
plt.xlabel('Tempo (ps)', fontsize=14,labelpad=10)
plt.ylabel('RMSD (Angstrom)', fontsize=14,labelpad=10)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show() 

## Monitoramento de ângulos diedrais

In [ ]:
# Cálculo dos ângulos diedrais ao longo da trajetória
diedros_implicit = pt.multidihedral(traj_implicit, resrange='1-6', dihedral_types="phi psi", dtype='dataframe')

In [ ]:
# Ângulos phi em solvente implícito
nrows, ncols = 3,2
cont = 0
plt.figure(figsize=(8, 10))
for i in range(nrows):
    for j in range(ncols):
        cont += 1
        lab = '$\phi_' + str(cont) + '$'
        lang = 'phi_'+ str(cont)
        if (cont > 1):
            plt.subplot(3, 2, cont - 1)
            plt.plot(diedros_implicit[lang], '-bo', markersize=2, linewidth=0)
            plt.xlabel('Tempo (em ps)', fontsize=14,labelpad=10)
            plt.ylabel(lab, fontsize=14)
plt.tight_layout()
plt.show()        

In [ ]:
# Ângulos psi em solvente implícito
nrows, ncols = 3,2
cont = 0
plt.figure(figsize=(8, 10))
for i in range(nrows):
    for j in range(ncols):
        cont += 1
        lab = '$\psi_' + str(cont) + '$'
        lang = 'psi_'+ str(cont)
        if (cont < 6):
            plt.subplot(3, 2, cont)
            plt.plot(diedros_implicit[lang], '-mo', markersize=2, linewidth=0)
            plt.xlabel('Tempo (em ps)', fontsize=14,labelpad=10)
            plt.ylabel(lab, fontsize=14)
plt.tight_layout()
plt.show()        

## Fração de Contatos Nativos

In [ ]:
# # Lendo o template e visualizando a estrutura
tpl = pt.load('ALA6helix.pdb')
v_tpl = nv.show_pytraj(tpl)
v_tpl.add_ball_and_stick()
v_tpl.add_cartoon()
v_tpl.parameters = dict(camera_type='perpective', background_color='black',fog_near=60, clip_dist=0)
v_tpl

In [ ]:
# Importar a biblioteca Pandas
import pandas as pd
print('Versão do Pandas', pd.__version__)

In [ ]:
# Calculando o número de contatos nativos da estrutra de referência, o template
n_cont_template = pt.native_contacts(tpl, ref=tpl, top='peptide.top', dtype='dataframe')
nc_max = n_cont_template.max()
print(nc_max)

In [ ]:
# Para calcular o número de contatos nativos durante a trajetória
n_cont = pt.native_contacts(traj_implicit, ref=tpl, top='peptide.top', dtype='dataframe')
n_cont.rename(columns={'Contacts_00001[native]': 'Native Contacts',                        'Contacts_00001[nonnative]': 'Non-Native Contacts'}, inplace=True)
fraction_native = n_cont['Native Contacts']/nc_max[0]
fraction_native
# OBS: os valores podem não coincidir pois se trata de trajetórias com pequeno grau de aleatoriedade.

In [ ]:
# Gráfico das frações de contatos nativos ao longo da trajetória
plt.figure(figsize=(10, 8))
plt.plot(fraction_native)
plt.title('Dinâmica Molecular da $(ALA)_6$ em solvente implícito', fontsize=16, pad = 14)
plt.xlabel('Tempo (ps)', fontsize=14,labelpad=10)
plt.ylabel('Fração de Contatos Nativos (%)', fontsize=14, labelpad=10)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()

## Considerando moléculas de solvente explicitamente

In [ ]:
# Montando a topologia para o sistema solvatado
# OBS: Não usar comentários na caixa que escreve o arquivo "leap_in_water.in"

In [ ]:
%%file leap_in_water.inp
source leaprc.protein.ff14SB
source leaprc.water.tip3p
set default PBRadii mbondi2
seq = sequence{NALA ALA ALA ALA ALA CALA}
charge seq
solvateBox seq TIP3PBOX 10.0
saveamberparm seq peptide_solv.top peptide_solv_ini.crd
savepdb seq peptide_solv_ini.pdb 
quit

In [ ]:
# Montando a topologia do sistema solvatado
!$AMBERHOME/bin/tleap -f leap_in_water.inp > leap_in_water.log

In [ ]:
# Escrevendo o input para otimizar a geometria do sistema solvatado
# OBS: Não usar comentários na caixa que escreve o arquivo "min_in_water.inp"

In [ ]:
%%file min_in_water.inp
Otm Geo
&cntrl
   imin=1, maxcyc=5000, ntpr=500, ntmin=1, ncyc=500, ntf=1, ntb=1, cut=12.0, igb=0,
&end

In [ ]:
# Comando para otimizar a geometria do sistema solvatado
!$AMBERHOME/bin/sander -O -i min_in_water.inp -p peptide_solv.top -c peptide_solv_ini.crd -r min_solv.rst7

In [ ]:
# Montando uma representação da estrutura otimizada do sistema peptídeo solvatado
ref_solv_min = pt.load('min_solv.rst7', top='peptide_solv.top')
view_min_solv = nv.show_pytraj(ref_solv_min)
view_min_solv.clear_representations()

In [ ]:
# Adicionando representações ao objeto view_min_solv e mostrando em tela
view_min_solv.add_ball_and_stick(selection='protein')
view_min_solv.add_ball_and_stick(selection='water', opacity=0.2)
view_min_solv.add_cartoon()
view_min_solv.parameters = dict(camera_type='perpective', background_color='black',fog_near=60, clip_dist=0)
view_min_solv

In [ ]:
# Escrevendo o input para dinâmica molecular do sistema solvatado
# OBS: Não usar comentários na caixa que escreve o arquivo "min_in_water.in"

In [ ]:
%%file md_in_water_NVT.inp
MD NVT em água
&cntrl
    nstlim=250000, dt=0.002, ntx=1, irest=0, ntpr=500, ntwr=500, ntwx=500, 
    ntt=3, tempi =250.0, temp0=300.0, tautp=2.0, ig=209858, gamma_ln=2.0, 
    cut=12.0, ntb=1, ntp=0, ntc=2, ntf=2, nrespa=2,
&end

In [ ]:
# Comando para executar a dinâmica molecular NVT no sistema solvatado
!$AMBERHOME/bin/sander -O -i md_in_water_NVT.inp -o md_in_water_NVT.out -p peptide_solv.top -c min_solv.rst7 -r md_in_water_NVT.rst -x md_in_water_NVT.nc

In [ ]:
# Carregando a trajetória do sistema solvatado
traj_in_water = pt.load('md_in_water_NVT.nc', top='peptide_solv.top')
traj_in_water